###**Head pose estimation** 

![picture](https://drive.google.com/uc?export=view&id=1sGZcJK_SKixlNjM-IeZSj-_D8yFB-B_L)


**Head pose estimation** is a computer vision technique that determines the position and orientation of a person's head in a three-dimensional space using image or video data. It involves detecting and tracking facial landmarks to estimate the rotation and translation of the head. It has applications in areas such as human-computer interaction, augmented reality, driver monitoring systems, and more.



## Area of usage

1. Improved facial recognition

2.   Surveillance and security

3. Surveillance and security

4. Gesture-based human-computer interaction

5. Realistic computer graphics

6. Immersive augmented reality

7. Driver monitoring for safety

8. Adaptive human-robot interaction

##Main task

![picture](https://drive.google.com/uc?export=view&id=1AwZUg4wosE4HA62JVmocPRdRAtbrfxGi)




### Existing approaches


![picture](https://drive.google.com/uc?export=view&id=1iA0DmiQFtTS6_nSNRLBzEWeZ7gLf9z0H)

1.   Two-Step Approach (usage of keypoints)
  

*  Use a 2D facial landmark tracker like Facial Alignment Network (**FAN**) or **Dlib** to identify the landmarks in the image and produce their coordinates.
*   Fit a 3D model of a human face to match these points to their 3D equivalents. A common approach is to fit a simple model of the mean human face. 



2.   One shot estimation
*  Use single model to estimate head pose. State-of-art approaches: **WHEnet**, **6DepNet** 

![picture](https://drive.google.com/uc?export=view&id=1A2eMLC6Vyybak723WeNOMKyiBthWoUah)




## To simplify task we focus on **YAW**, **PITCH** angles estimation

Proposed method:

![picture](https://drive.google.com/uc?export=view&id=1ZI8VjDf1BhG-lVMohnbvS60Qb06heJFZ)


### Existing datasets



1.   BIWI Kinect Head Pose
2.   AFLW2000
3.   300W_LP
4.   AFL
5. IBUG
6. HELEN


**300W_LP** dataset  contains almost all other datasets. We use it for training

![picture](https://drive.google.com/uc?export=view&id=1vcVO81hH6hQJHDaiTD29XrCD_ziV4xLN)



## Main steps elements to run DL model for head pose prediction:

*   dataset
*   model
*   train
*   validation
*   demo



Dataset generation

1. Need prepare utils

In [1]:
!pip install open3d

ERROR: Could not find a version that satisfies the requirement open4d (from versions: none)
ERROR: No matching distribution found for open4d


In [ ]:
import numpy as np
import scipy.io as sio
import open3d as o3d
import open3d.visualization.gui as gui
import pywavefront
from astropy.coordinates import cartesian_to_spherical, spherical_to_cartesian
from scipy.spatial.transform import Rotation as R
import cv2

ModuleNotFoundError: ignored

Helper function to create pose from W300_LP mat

In [ ]:
def get_pose_W300_LP_from_mat(mat_path):
    mat = sio.loadmat(mat_path)
    # mat is constructed: pitch, yaw, roll, tdx, tdy, tdz, scale_factor
    pre_pose_params = mat['Pose_Para'][0]
    # Get pitch, yaw, roll
    pose_params = pre_pose_params[:3]
    return pose_params

For visualization we use mesh in format obj. 
Read mesh using: 
```
pywavefront
```

For 3d mesh representation it is nice to use lib:


```
open3d
```




In [ ]:
def setupMesh(path):
    # Read 3d mesh from obj. file
    meshObj = pywavefront.Wavefront(path, collect_faces=True)

    # Build open3D mesh from pywavefront
    mesh = o3d.geometry.TriangleMesh()
    vertices = np.array(meshObj.vertices, dtype=np.float32)
    triangles = np.array(meshObj.meshes['eye_low_L_eyeball_mesh.002'].faces).astype(np.uint32)
    mesh.vertices = o3d.utility.Vector3dVector(vertices)
    mesh.triangles = o3d.utility.Vector3iVector(triangles)
    #Calculate normals for better visualization
    mesh.compute_vertex_normals()
    return mesh

We need to have helper function for conversion from:

```
yaw,pitch,roll -> direction vector ▶ direction vector->yaw,pitch,roll
```



In [ ]:
def convertAngle2Direction(yaw, pitch):
    rho = 1.0
    x, y, z = spherical_to_cartesian(rho, pitch, yaw)
    return np.array([x, y, z], dtype=np.float32)

def convertDirection2angle(x, y, z):
    direction = np.array([x, y, z], dtype=np.float32)
    normalized_direction = direction / np.linalg.norm(direction)
    rho, theta, phi = cartesian_to_spherical(normalized_direction[0], normalized_direction[1], normalized_direction[2])
    yaw = phi.radian * 180 / np.pi
    pitch = theta.radian * 180 / np.pi
    return yaw, pitch

One imortant thing - **visualization**

For that we use **open3d** wrapped inside class **HeadPose3DVisualizer**

We need to:

*  Render angles
*  Render directions




In [ ]:
class HeadPose3DVisualizer():
    def __init__(self, w, h, mesh_path):
        self.width = w
        self.height = h
        self.data_mesh = setupMesh(mesh_path)

        # Need to for 3d rendering
        self.prev_yaw = 0
        self.prev_pitch = 0
        self.prev_roll = 0

    def initMaterial(self):
        # Initialize 3d mesh material
        self.material = o3d.visualization.rendering.MaterialRecord()
        self.material.base_color = (0.2, 0.3, 0.3, 1.0)
        self.material.shader = "defaultLit"
        self.bg_color = np.ndarray((4, 1), dtype=np.float32)

    def renderDirection(self, x, y, z):
        yaw, pitch = convertDirection2angle(x,y,z)
        return self.renderAngles(yaw, pitch, 0.0)

    def renderAngles(self, yaw, pitch, roll):
        render = o3d.visualization.rendering.OffscreenRenderer(self.width, self.height)
        self.initMaterial()
        # Need to compensate rotation
        cur_rotation = R.from_euler('yxz', [-self.prev_yaw, self.prev_pitch, -self.prev_roll], degrees=True)
        self.data_mesh.rotate(cur_rotation.as_matrix(), center=(0, 0, 0))
        self.data_mesh.rotate(R.from_euler('zxy', [roll, -pitch, yaw], degrees=True).as_matrix(), center=(0, 0, 0))

        render.scene.scene.add_geometry(name="geo2", geometry=self.data_mesh, material=self.material)
        self.prev_yaw = yaw
        self.prev_pitch = pitch
        self.prev_roll = roll

        #Camera setup
        vertical_field_of_view = 75.0  # between 5 and 90 degrees
        aspect_ratio = 1.0  # azimuth over elevation
        near_plane = 0.10000000149011612
        far_plane = 1.1079597473144531
        fov_type = o3d.visualization.rendering.Camera.FovType.Horizontal
        render.scene.camera.set_projection(vertical_field_of_view, aspect_ratio, near_plane, far_plane, fov_type)
        center = [0, 0, 0]  # look_at target
        eye = [0, 0.02, 0.33]  # camera position
        up = [0, 1, 0]  # camera orientation
        render.scene.camera.look_at(center, eye, up)
        # render to image
        img_o3d = render.render_to_image()
        img_cv2 = cv2.cvtColor(np.array(img_o3d), cv2.COLOR_RGBA2BGRA)

        return img_cv2[:, :, 0:3]

Now we can switch to the **dataset**

For that  **torch** has nice classes:


```
torch.utils.data.dataset.Dataset
torch.utils.data.DataLoader
torchvision.transforms
```

Firstly implement basic class inhereted from `torch.utils.data.dataset.Dataset`
For that we need implement several methods: `__getitem__`,  `__len__`




In [ ]:
def getData(path):
    x_data = []
    y_data = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if '.mat' not in file:
                mat_path = os.path.join(root, file.split('.')[0])
                img_path = os.path.join(root, file)
                x_data.append(img_path)
                y_data.append(mat_path)

    return x_data, y_data

class PoseDataW300_LP(Dataset):
    def __init__(self, data_dir, transform):
        self.data_dir = data_dir
        self.transform = transform
        self.x_data, self.y_data = getData(data_dir)
        self.data_length = len(self.x_data)


Method `__getitem__` and `__len__`
Inside `__getitem__` we read our data: image and mat label. 

In [ ]:
    def __getitem__(self, index):
        # Load image
        img = io.imread(self.x_data[index])
        # Load label
        mat_path =self.y_data[index]


Now using ```get_pose_W300_LP_from_mat``` we get our pose.
With  ```convertAngle2Direction(yaw, pitch)``` we transform angles to direction vector





In [ ]:

        # Get pitch, yaw, roll from mat
        pose = get_pose_W300_LP_from_mat(mat_path)
        # Convert radians to degrees.
        pitch = pose[0]
        yaw = -pose[1]
        roll = pose[2]

        #Convert yaw, pitch to direction vector
        direction_vector = convertAngle2Direction(yaw, pitch)
        sample = {'image': img, 'direction': direction_vector}

        if self.transform is not None:
            return self.transform(sample)

        return sample

    def __len__(self):
        return self.data_length

Now we have our own data class. 

But how about adding some augmentation?
Here we  can use `Rotation` augmentation. 
Each augmentation should have implemented ```__call__``` method

We would like to rotate image + direction vector into angle from some range **[minAngle, maxAngle]**. For example better to use **[-30, 30]** degrees



In [ ]:
class Rotate(object):
    #Rotate image with direction vector
    def __init__(self, angle_range):
        self.angle_range = angle_range
        assert isinstance(angle_range, (float, tuple))

    def __call__(self, sample):
        image, direction = sample['image'], sample['direction']

        angle = random.randint(self.angle_range[0], self.angle_range[1])
        rotater = transforms.RandomRotation(degrees=[angle, angle])
        yaw, pitch = convertDirection2angle(direction[0], direction[1], direction[2])
        rotation = RotationObject.from_euler('zxy', [0.0, -pitch, yaw], degrees=True)
        vect = [0, 0, -angle * np.pi / 180.0]

        newRotation = RotationObject.from_rotvec(vect)
        mainRotation = newRotation.as_matrix().dot(rotation.as_matrix())
        matrixRot = RotationObject.from_matrix(mainRotation)

        processedAngles = matrixRot.as_euler(seq='zxy', degrees=True)
        new_yaw = processedAngles[2] * np.pi/180.0
        new_pitch = processedAngles[1] * np.pi/180.0
        return {'image': rotater(image), 'direction': convertAngle2Direction(new_yaw, new_pitch)}
        

Transforms numpy image, tuple -> tensors

In [ ]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, landmarks = sample['image'], sample['direction']
        return {'image': torch.from_numpy(image),
                'direction': torch.from_numpy(landmarks)}

Now we are ready to run dataset generation and see how it looks like
For that we need:


1.   Create our dataset object:```PoseDataW300_LP```

2.   Add data loader for generating batches ```DataLoader```
3. Add visualization with ```HeadDirectionVisualizer```



In [ ]:
if __name__ == '__main__':
    head_pose_dataset = PoseDataW300_LP(data_dir='D:/Workshop/Dataset/300W_LP/AFW',
                                        transform =transforms.Compose([Rotate((-30, 30)),ToTensor()]))

    visualizer = HeadDirectionVisualizer(1024, 512, "D:/Workshop/mesh/mesh.obj")

    dataloader = DataLoader(head_pose_dataset, batch_size=4,
                            shuffle=True, num_workers=4)


    for i_batch, sample_batched in enumerate(dataloader):
        print(i_batch, sample_batched['image'].size(), sample_batched['direction'].size())
        image = sample_batched['image'][0]
        direction = sample_batched['direction'][0].detach().cpu().numpy()
        np_image = np.array(image)[..., :3]
        img = o3d.geometry.Image(np_image)
        result = visualizer.showDirection(img, direction[0], direction[1], direction[2])



![picture](https://drive.google.com/uc?export=view&id=1nMk6KwH0R_Gkve8qtw8X4MktoXHNIpMn)


## Now we can switch to the model creation
As basic model we will use AlexNet NN with several modifications:


1.   Add  `nn.AdaptiveAvgPool2d` at the and of feature extractior
2.  Regression layers will have less parameters **4096 -> 256**
3.  Last layer has tensor with 3 velues - **direction vector**

![picture](https://drive.google.com/uc?export=view&id=1jmLuDlA9VifWzUtH6zn5DRIj_DRUsAcD)



In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.autograd import Variable
class HeadPOseEstimator(nn.Module):
    def __init__(self):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(64),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # output: 128 x 8 x 8
            nn.BatchNorm2d(128),

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),

            nn.MaxPool2d(2, 2),  # output: 256 x 4 x 4
            nn.BatchNorm2d(256),

            nn.AdaptiveAvgPool2d((1, 1)),  # 256 x 1 x 1
            )

        self.regression = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 256),
            nn.Linear(256, 256),
            nn.Linear(256, 3))


    def forward(self, x):
        x = self.features(x)
        h = x.view(x.shape[0], -1)
        x = self.regression(h)
        return x, h

When we have a **model** and **data** - need to train!

To compare 2 direction vectors better to use **CosineSimilarity** instead of **MSE**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch

regression_loss = nn.CosineSimilarity(dim=1, eps=1e-6)
outputs = torch.rand(4, 3)
gt_data = torch.rand(4, 3)


print(outputs)
print(gt_data)

loss = regression_loss(outputs, gt_data)
print('Total loss for batch: {}'.format(torch.mean(loss)))

Need to define optimizar. As mentioned before - we can use` Momentum,Adam, RMSProp` and other optimizers. In our case we will use **Adam** optimizer

In [ ]:
    model = HeadPOseEstimator()

    # Optimizers specified in the torch.optim package
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

Training loop

1. `DataLoader`->Data

2. Zeros the optimizer’s gradients

3. Inference on model

4. Calculates the loss for that set of predictions vs. the labels on the dataset

5. Calculates the backward gradients over the learning weights

6. Conduct one learning step

7. Report loss


In [ ]:
    head_pose_dataset_train = PoseDataW300_LP(data_dir='D:/Workshop/Dataset/300W_LP/LFPW',
                                        transform =transforms.Compose([ToTensor()]))

    head_pose_dataset_validation = PoseDataW300_LP(data_dir='D:/Workshop/Dataset/300W_LP/IBUG',
                                        transform=transforms.Compose([ToTensor()]))

    visualizer = HeadDirectionVisualizer(1024, 512, "D:/Workshop/mesh/mesh.obj")

    dataloader_train = DataLoader(head_pose_dataset_train, batch_size=16,
                            shuffle=True, num_workers=4)

    dataloader_validation = DataLoader(head_pose_dataset_validation, batch_size=16,
                                  shuffle=True, num_workers=4)

    def train_one_epoch(epoch_index, tb_writer):
        running_loss = 0.
        last_loss = 0.
        for i, data in enumerate(dataloader_train):
            inputs, directions = data
            # Zero your gradients
            optimizer.zero_grad()
            # Make predictions
            outputs = model(inputs)
            # Compute the loss and its gradients
            loss = loss_function(outputs, directions)
            loss.backward()
            # Adjust learning weights
            optimizer.step()

            # Gather data and report
            running_loss += loss.item()
            if i % 1000 == 999:
                last_loss = running_loss / 1000  # loss per batch
                print('  batch {} loss: {}'.format(i + 1, last_loss))
                tb_x = epoch_index * len(dataloader) + i + 1
                tb_writer.add_scalar('Loss/train', last_loss, tb_x)
                running_loss = 0.

        return last_loss


    # Initializing in a separate cell so we can easily add more epochs to the same run
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
    epoch_number = 0

    EPOCHS = 5

    best_vloss = 1_000_000.

    for epoch in range(EPOCHS):
        print('EPOCH {}:'.format(epoch_number + 1))

        # Make sure gradient tracking is on, and do a pass over the data
        model.train(True)
        avg_loss = train_one_epoch(epoch_number, writer)

        running_vloss = 0.0
        # Set the model to evaluation mode, disabling dropout and using population
        # statistics for batch normalization.
        model.eval()

        # Disable gradient computation and reduce memory consumption.
        with torch.no_grad():
            for i, vdata in enumerate(dataloader_validation):
                vinputs, vlabels = vdata
                voutputs = model(vinputs)
                vloss = regression_loss(voutputs, vlabels)
                running_vloss += vloss

        avg_vloss = running_vloss / (i + 1)
        print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

        # Log the running loss averaged per batch
        # for both training and validation
        writer.add_scalars('Training vs. Validation Loss',
                           {'Training': avg_loss, 'Validation': avg_vloss},
                           epoch_number + 1)
        writer.flush()

        # Track best performance, and save the model's state
        if avg_vloss < best_vloss:
            best_vloss = avg_vloss
            model_path = 'model_{}_{}'.format(timestamp, epoch_number)
            torch.save(model.state_dict(), model_path)

        epoch_number += 1